In [2]:
import re
import pandas as pd
import time
from collections import Counter
import numpy as np
import nltk 
nltk.data.path.append("/var/lib/docker/nltk_data")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from textblob import Word
from pywsd.utils import lemmatize_sentence
import dask.dataframe as dd
import dask
from dask.diagnostics import ProgressBar

Warming up PyWSD (takes ~10 secs)... took 3.375617504119873 secs.


In [8]:
data=dd.read_csv('data/ads_en_us.csv',engine = "python",encoding='utf-8',header=0,dtype='object',error_bad_lines=False)

Skipping line 176: unexpected end of data


In [9]:
data

,Unnamed: 0,catid,id,subcatid,value
npartitions=5,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [14]:
pbar = ProgressBar()
pbar.register()
lemmatizer = WordNetLemmatizer()

In [15]:
def clean_str(s):
    """Clean sentence"""
    s=str(s)
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\(", " \( ", s)
    s = re.sub(r"\)", " \) ", s)
    s = re.sub(r"\?", " \? ", s)
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r'\S*(x{2,}|X{2,})\S*',"xxx", s)
    s = re.sub(r'[^\x00-\x7F]+', "", s)
    s = re.sub(r'[^\w\s]',"",s)
    return s.strip().lower()

In [16]:
def perform_lemm(sentence):
    word_tokens = word_tokenize(sentence)
    lemm_sentence = [lemmatizer.lemmatize(w,pos='v') for w in word_tokens]
    return ' '.join(lemm_sentence)

In [17]:
data = data.drop_duplicates(subset='value',keep='last')
data.compute()

[                                        ] | 0% Completed |  1.9s

Skipping line 10917: unexpected end of data
Skipping line 12891: unexpected end of data
Skipping line 18: Expected 5 fields in line 18, saw 6


[########                                ] | 20% Completed |  2.7s

Skipping line 11426: unexpected end of data
Skipping line 12755: unexpected end of data
Skipping line 11943: unexpected end of data


[##############                          ] | 35% Completed |  3.2s

Skipping line 11744: unexpected end of data
Skipping line 12046: unexpected end of data


[########################################] | 100% Completed |  3.4s


,Unnamed: 0,catid,id,subcatid,value
1,1,2,45499,27,Overview \n \nWhy AFFIRMA Rehabilitation? Our ...
2,2,2,45522,27,Overview:\n\nUnder general supervision by the ...
15,15,2,45563,27,A California group is currently seeking a locu...
57,57,2,45622,21,Job Description \n\n Job Description \n\nI am ...
101,101,2,45671,27,"Assesses, plans, implements, coordinates, moni..."
104,104,2,45675,27,"Assists the Pharmacist in the preparation, dis..."
106,106,2,45679,134,Procurement Contracts Manager \n\n \n\n The Op...
107,107,2,45683,16,"Director, Technical Accounting and Revenue Rec..."
113,113,2,45694,34,Software Engineering Lead/Architect \n\nAs a S...
118,118,2,45712,134,Be your own boss working under a Strong Brand ...


In [18]:
data

,Unnamed: 0,catid,id,subcatid,value
npartitions=1,,,,,
,object,object,object,object,object
,...,...,...,...,...


In [8]:

data_27 = data.loc[data.subcatid == 27, :]
data_27s = data_27.sample(frac=0.5)
data1 = data[data.subcatid != 27]
data_s = dd.concat([data1, data_27s], axis = 0 )
#data_bal=data_s.groupby('subcatid').filter(lambda x: len(x) >= 100)
data_s = data_s.repartition(npartitions=16) 
data_s.compute()

Skipping line 24471: unexpected end of data
Skipping line 24802: unexpected end of data
Skipping line 22652: unexpected end of data
Skipping line 22947: unexpected end of data


,Unnamed: 0,catid,id,subcatid,value
1,1,2,45499,27,Overview \n \nWhy AFFIRMA Rehabilitation? Our ...
2,2,2,45522,27,Overview:\n\nUnder general supervision by the ...
15,15,2,45563,27,A California group is currently seeking a locu...
57,57,2,45622,21,Job Description \n\n Job Description \n\nI am ...
101,101,2,45671,27,"Assesses, plans, implements, coordinates, moni..."
104,104,2,45675,27,"Assists the Pharmacist in the preparation, dis..."
106,106,2,45679,134,Procurement Contracts Manager \n\n \n\n The Op...
107,107,2,45683,16,"Director, Technical Accounting and Revenue Rec..."
113,113,2,45694,34,Software Engineering Lead/Architect \n\nAs a S...
118,118,2,45712,134,Be your own boss working under a Strong Brand ...


In [ ]:
%%time
data_s['value_processed'] = data_s.value.apply(clean_str)
data_s['value_processed_lemm'] = data_s.value_processed.apply(perform_lemm)
data_s.compute()

[#                                       ] | 4% Completed |  2.6s

Skipping line 22947: unexpected end of data


[##                                      ] | 6% Completed |  2.8s

Skipping line 24802: unexpected end of data
Skipping line 22652: unexpected end of data
Skipping line 24471: unexpected end of data


[###########################             ] | 68% Completed |  2min 40.5s

In [65]:
%%time
data_s.value_processed_lemm.head(20)

[######                                  ] | 16% Completed |  2.6s

Skipping line 22947: unexpected end of data
Skipping line 24471: unexpected end of data
Skipping line 24802: unexpected end of data


[##########                              ] | 25% Completed |  2.9s

Skipping line 22652: unexpected end of data


[########################################] | 100% Completed | 22.4s
CPU times: user 22.5 s, sys: 410 ms, total: 22.9 s
Wall time: 22.4 s


1      overview why affirma rehabilitation our innova...
2      overview under general supervision by the supe...
15     a california group be currently seek a locum t...
57     job description job description i be please to...
101    assess plan implement coordinate monitor and e...
104    assist the pharmacist in the preparation dispe...
106    procurement contract manager the opportunity m...
107    director technical account and revenue recogni...
113    software engineer lead architect as a software...
118    be your own boss work under a strong brand wit...
119    usa truck dedicate service division have truck...
120    who be cynergistek cynergistek be a top rank c...
121    we have an open for 7 full and part timewaiter...
122    department mml specialty sales a life change c...
123    a delaware base urgent care facility be curren...
124    job description director technical account and...
125    job description enterprise sales development r...
126    this position do require

In [66]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer ,TfidfTransformer
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.externals.joblib import Parallel, delayed
import dask_ml.model_selection as dcv

In [67]:
print(data.shape)

(Delayed('int-66391328-847b-46d8-84ee-1e0659a7a902'), 7)


In [70]:
X = data.value_processed_lemm.values
y = data.subcatid.values
X.compute()
X_train, X_test, y_train, y_test = dcv.train_test_split(X, y, test_size=0.3, random_state = 5)

[#####                                   ] | 13% Completed |  2.8s

Skipping line 24471: unexpected end of data
Skipping line 22652: unexpected end of data
Skipping line 24802: unexpected end of data


[#####################                   ] | 53% Completed |  3.2s

Skipping line 22947: unexpected end of data


[#############################           ] | 73% Completed | 39.4s


TypeError: expected string or bytes-like object

In [14]:
"""
# Number of trees in random forest
"""
n_estimators = [200,300]
"""
# Maximum number of levels in tree
"""
max_depth =[150,200]
"""
#max_depth.append(None)
# Minimum number of samples required to split a node
"""
min_samples_split = [10,15]
"""
# Minimum number of samples required at each leaf node
"""
min_samples_leaf = [1]

"""
# Create the random grid
"""
random_grid = {
               'clf__n_estimators': n_estimators,
               'clf__max_depth': max_depth,
               'clf__min_samples_split': min_samples_split,
               'clf__min_samples_leaf': min_samples_leaf
              }

#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', RandomForestClassifier(max_features='auto',bootstrap=False,random_state = 42))])

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'